In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/test_features.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv


In [10]:
!rm train_targets_folds.csv
!wget https://github.com/Mayukhdeb/moa-classification-with-model-blending/raw/main/folds/train_targets_folds.csv

rm: cannot remove 'train_targets_folds.csv': No such file or directory
--2020-10-06 06:36:48--  https://github.com/Mayukhdeb/moa-classification-with-model-blending/raw/main/folds/train_targets_folds.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Mayukhdeb/moa-classification-with-model-blending/main/folds/train_targets_folds.csv [following]
--2020-10-06 06:36:48--  https://raw.githubusercontent.com/Mayukhdeb/moa-classification-with-model-blending/main/folds/train_targets_folds.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.248.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.248.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10182351 (9.7M) [text/plain]
Saving to: ‘train_targets_folds.csv’

train_targets_folds 100%[===========

In [11]:
!pip install iterative-stratification

In [12]:
pd.read_csv("./train_targets_folds.csv").head()

,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,kfold
0,id_95029ba69,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
1,id_937bf8e8f,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
2,id_59e0b53af,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
3,id_1b0b65599,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,id_1d650ed9d,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler

import numpy as np 
import pandas as pd 

from IPython.display import clear_output
import random 
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter1d   ## smoother
from tqdm.notebook import tqdm, tnrange
import os
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

plt.rcParams['figure.figsize'] = 15, 7

CGREEN  = '\33[32m'
CBLUE =  '\033[34m'
CRED = '\033[1;31m'
CEND  = '\33[0m'

def seed_everything(seed=1903):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    print("seed kar diya")

In [14]:
if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'
    
device


'cuda'

In [15]:

def preprocess(df, select_features):
    df = df.copy()
    top_features = [ 0, 1, 2,   3,   4,   5,   6,   7,   9,  11,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  29,  30,  31,
        32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,  60,
        61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
        74,  75,  76,  78,  79,  80,  81,  82,  83,  84,  86,  87,  88,
        89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
       115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197,
       198, 199, 200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 217, 218, 219, 220, 221, 223, 224, 225, 226,
       227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
       240, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253,
       254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266,
       267, 268, 269, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280,
       281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 294,
       295, 296, 298, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,
       310, 311, 312, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323,
       324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336,
       337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349,
       350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362,
       363, 364, 365, 366, 367, 368, 369, 370, 371, 374, 375, 376, 377,
       378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 390, 391,
       392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404,
       405, 406, 407, 408, 409, 411, 412, 413, 414, 415, 416, 417, 418,
       419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431,
       432, 434, 435, 436, 437, 438, 439, 440, 442, 443, 444, 445, 446,
       447, 448, 449, 450, 453, 454, 456, 457, 458, 459, 460, 461, 462,
       463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475,
       476, 477, 478, 479, 481, 482, 483, 484, 485, 486, 487, 488, 489,
       490, 491, 492, 493, 494, 495, 496, 498, 500, 501, 502, 503, 505,
       506, 507, 509, 510, 511, 512, 513, 514, 515, 518, 519, 520, 521,
       522, 523, 524, 525, 526, 527, 528, 530, 531, 532, 534, 535, 536,
       538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 549, 550, 551,
       552, 554, 557, 559, 560, 561, 562, 565, 566, 567, 568, 569, 570,
       571, 572, 573, 574, 575, 577, 578, 580, 581, 582, 583, 584, 585,
       586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 599,
       600, 601, 602, 606, 607, 608, 609, 611, 612, 613, 615, 616, 617,
       618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630,
       631, 632, 633, 634, 635, 636, 637, 638, 639, 641, 642, 643, 644,
       645, 646, 647, 648, 649, 650, 651, 652, 654, 655, 656, 658, 659,
       660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672,
       673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685,
       686, 687, 688, 689, 691, 692, 693, 694, 695, 696, 697, 699, 700,
       701, 702, 704, 705, 707, 708, 709, 710, 711, 713, 714, 716, 717,
       718, 720, 721, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732,
       733, 734, 735, 737, 738, 739, 740, 742, 743, 744, 745, 746, 747,
       748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 759, 760, 761,
       762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774,
       775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788,
       789, 790, 792, 793, 794, 795, 796, 797, 798, 800, 801, 802, 803,
       804, 805, 806, 808, 809, 811, 813, 814, 815, 816, 817, 818, 819,
       821, 822, 823, 825, 826, 827, 828, 829, 830, 831, 832, 834, 835,
       837, 838, 839, 840, 841, 842, 845, 846, 847, 848, 850, 851, 852,
       854, 855, 856, 858, 859, 860, 861, 862, 864, 866, 867, 868, 869,
       870, 871, 872, 873, 874]
    all_columns = df.columns

    if select_features == True:
        df=df[all_columns[top_features]]
    else:
        pass
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({ 24: 0, 48: 1, 72: 2})
    
    return df

In [16]:
class MoaDataset(Dataset):
    def __init__(self,features,targets):
        
        self.features  = features
        self.targets = targets
        
    def sizes(self):
        print("features size = ", self.features.shape[1])
        print("targets size = ", self.targets.shape[1])
        
    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, item):
        return {
            "x": torch.tensor(self.features[item, :], dtype=torch.float),
            "y": torch.tensor(self.targets[item, :], dtype=torch.float)
        }

In [25]:

class Engine:
      def __init__(self, model, optimizer, device, scheduler):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler

      @staticmethod
      def loss_fn(targets, outputs):
        return nn.BCEWithLogitsLoss()(outputs, targets)

      def train(self, data_loader):
        self.model.train()
        final_loss=0
        for data in data_loader:
            self.optimizer.zero_grad()
            inputs = data["x"].to(self.device)
            targets = data["y"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            loss.backward()
            
            self.optimizer.step()
            
            final_loss += loss.item()
        self.scheduler.step(1.)
        return(final_loss / len(data_loader))


      def evaluate(self, data_loader):
        self.model.eval()
        final_loss=0
        for data in data_loader:
            #self.optimizer.zero_grad()
            inputs = data["x"].to(self.device)
            targets = data["y"].to(self.device)
            outputs = self.model(inputs)
            loss = self.loss_fn(targets, outputs)
            #loss.backward()
            #self.optimizer.step()
            final_loss += loss.item()
        return(final_loss / len(data_loader))

In [26]:
def run_training(model_class ,fold, save_model=True, select_features = False, model_id = 0):
    EPOCHS = 100
    df = pd.read_csv('../input/lish-moa/train_features.csv')
    df = preprocess(df, select_features = select_features)
    targets_df = pd.read_csv('./train_targets_folds.csv')
    
    feature_columns = df.drop("sig_id", axis=1).columns
    target_columns = targets_df.drop(["sig_id", "kfold"], axis=1).columns
    

    df = df.merge(targets_df, on="sig_id", how="left")
    print(df.shape)
    df = df[df.kfold != "hold"]
    print(df.shape)

    train_df = df[df.kfold != str(fold)].reset_index(drop=True)
    valid_df = df[df.kfold == str(fold)].reset_index(drop=True)

    
    x_train = train_df[feature_columns].to_numpy()
    y_train = train_df[target_columns].to_numpy()
    
    x_val = valid_df[feature_columns].to_numpy()
    y_val = valid_df[target_columns].to_numpy()
    
    train_dataset = MoaDataset(features = x_train, targets=y_train)
    valid_dataset = MoaDataset(features = x_val, targets=y_val)
    
    train_loader = DataLoader(
        train_dataset, batch_size=128, shuffle=True
    )
    
    val_loader = DataLoader(
        valid_dataset, batch_size=1024, shuffle=True
    )
    
    model = model_class()
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), 
                                       lr = 0.004299882049752947, 
                                       weight_decay=2e-5)
    
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                 mode='min', 
                                                 factor=0.1, ## wooo hoo
                                                 patience=7, ## was 3 for 158 
                                                 eps=1e-4, 
                                                 verbose=True)
    criterion = nn.BCEWithLogitsLoss()
    eng = Engine(model, optimizer, device='cuda', scheduler = scheduler)
    
    best_loss = 999
    early_stop_iter = 15
    early_stop_count=0

    for epoch in tnrange(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.evaluate(val_loader)
        #print("train_loss:", train_loss, "val_loss:", valid_loss)
        if valid_loss<best_loss:
            best_loss = valid_loss
            if save_model:
                torch.save(model.state_dict(), "model_"+str(fold + model_id) +".pth")
                pass
        else:
            early_stop_count +=1
            
        if early_stop_count>early_stop_iter:
            break
                    
    return(best_loss)


In [27]:

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(785)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(785, 2048))
        
        self.batch_norm2 = nn.BatchNorm1d(2048)
        self.dropout2 = nn.Dropout(0.5)
        self.dense2 = nn.utils.weight_norm(nn.Linear(2048, 1048))
        
        self.batch_norm3 = nn.BatchNorm1d(1048)
        self.dropout3 = nn.Dropout(0.5)
        self.dense3 = nn.utils.weight_norm(nn.Linear(1048, 206))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)*2
        return x

In [28]:
best_losses = []
num_folds = 7
for fold in range(0, num_folds):
    print(best_losses)
    best_losses.append(run_training(model_class = Model, fold=fold, select_features = True))

[]
(23814, 993)
(20837, 993)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014750239128867785]
(23814, 993)
(20837, 993)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014750239128867785, 0.0148788100729386]
(23814, 993)
(20837, 993)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014750239128867785, 0.0148788100729386, 0.01494291021178166]
(23814, 993)
(20837, 993)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014750239128867785, 0.0148788100729386, 0.01494291021178166, 0.014640144382913908]
(23814, 993)
(20837, 993)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014750239128867785, 0.0148788100729386, 0.01494291021178166, 0.014640144382913908, 0.014733383742471537]
(23814, 993)
(20837, 993)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.014750239128867785, 0.0148788100729386, 0.01494291021178166, 0.014640144382913908, 0.014733383742471537, 0.014745427606006464]
(23814, 993)
(20837, 993)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.



In [29]:

class Model_2(nn.Module):
    def __init__(self):
        super(Model_2, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(875)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(875, 2048))  ## was 948 and 2 layers ofr best yet 128
        

        
        self.batch_norm3 = nn.BatchNorm1d(2048)
        self.dropout3 = nn.Dropout(0.5)
        self.dense3 = nn.utils.weight_norm(nn.Linear(2048, 206))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.leaky_relu(self.dense1(x))

        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

In [30]:
best_losses_2 = []
num_folds = 7
for fold in range(num_folds):
    print(best_losses_2)
    best_losses_2.append(run_training(model_class = Model_2, fold=fold, select_features = False, model_id = 100))

[]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.01504549446205298]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.01504549446205298, 0.015145722155769667]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.01504549446205298, 0.015145722155769667, 0.015178757098813852]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.01504549446205298, 0.015145722155769667, 0.015178757098813852, 0.01497190756102403]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.01504549446205298, 0.015145722155769667, 0.015178757098813852, 0.01497190756102403, 0.015084310434758663]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.

[0.01504549446205298, 0.015145722155769667, 0.015178757098813852, 0.01497190756102403, 0.015084310434758663, 0.01505639310926199]
(23814, 1083)
(20837, 1083)


Epoch     9: reducing learning rate of group 0 to 4.2999e-04.
Epoch    17: reducing learning rate of group 0 to 4.2999e-05.



In [31]:
print(best_losses_2)  ## better than old model
np.array(best_losses_2).mean()

[0.01504549446205298, 0.015145722155769667, 0.015178757098813852, 0.01497190756102403, 0.015084310434758663, 0.01505639310926199, 0.015355108305811882]


0.015119670446784724

In [64]:
!zip -r models.zip *.pth 

  adding: model_0.pth (deflated 5%)
  adding: model_1.pth (deflated 5%)
  adding: model_100.pth (deflated 6%)
  adding: model_101.pth (deflated 6%)
  adding: model_102.pth (deflated 6%)
  adding: model_103.pth (deflated 6%)
  adding: model_104.pth (deflated 6%)
  adding: model_105.pth (deflated 6%)
  adding: model_106.pth (deflated 6%)
  adding: model_2.pth (deflated 5%)
  adding: model_3.pth (deflated 5%)
  adding: model_4.pth (deflated 5%)
  adding: model_5.pth (deflated 5%)
  adding: model_6.pth (deflated 5%)


In [32]:
def load_fold(fold, select_features = False):

    df = pd.read_csv('../input/lish-moa/train_features.csv')
    df = preprocess(df, select_features = select_features)
    targets_df = pd.read_csv('./train_targets_folds.csv')

    feature_columns = df.drop("sig_id", axis=1).columns
    target_columns = targets_df.drop(["sig_id", "kfold"], axis=1).columns


    df = df.merge(targets_df, on="sig_id", how="left")

    train_df = df[df.kfold != str(fold)].reset_index(drop=True)
    valid_df = df[df.kfold == str(fold)].reset_index(drop=True)

    x_train = train_df[feature_columns].to_numpy()
    y_train = train_df[target_columns].to_numpy()

    x_val = valid_df[feature_columns].to_numpy()
    y_val = valid_df[target_columns].to_numpy()

    return x_train, y_train, x_val, y_val

class blend():
    def __init__(self,all_preds_np):
        
        self.all_preds = all_preds_np
           
    def predict(self, weights):
        self.weights = weights
        final_pred = np.zeros_like(self.all_preds[0])
        
        for i in range(len(self.all_preds)):
            final_pred += self.all_preds[i] * self.weights[i]
            
        final_pred = final_pred/self.weights.sum()
        
        return final_pred


def inference_fn(model, test_features, device):

    results = np.zeros([test_features.shape[0], 206])
    test_features_tensor = torch.tensor(test_features).float().to(device)
    rows = model(test_features_tensor).sigmoid().detach().cpu().numpy()
    results = rows 

    return results 

class pytorch_model():
    def __init__(self, model_class, model_path, device):
        
        self.model_class = model_class
        self.model_path = model_path
        self.device = device
        
        self.model = self.model_class()
        if self.device == "cuda":
            self.model.load_state_dict(torch.load(self.model_path))
        else:
            self.model.load_state_dict(torch.load(self.model_path, map_location=torch.device('cpu') ))

        self.model.to(self.device)
        self.model.eval()
        
    def predict(self, x):
        
        pred = self.model(x)
        
        return pred 
    
    def __call__(self, x):
        return self.predict(x)


def log_loss_metric(y_true, y_pred):
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = - np.mean(np.mean(y_true * np.log(y_pred_clip) + (1 - y_true) * np.log(1 - y_pred_clip), axis = 1))
    return loss

In [62]:
x_train, y_train, x_val, y_val = load_fold("hold", select_features = True)

all_preds = []
for i in range(7):
    m1  = pytorch_model(Model, "model_"+ str(i) +".pth", device)
    p1 = inference_fn(m1, x_val, device)
    l1 = log_loss_metric(y_val, p1)
    print(l1)
    all_preds.append(p1)

0.014723096311326197
0.014694604602326043
0.014695792870155069
0.014690191821428644
0.01466539651926048
0.014683284464815906
0.014654086652227317


In [63]:
x_train, y_train, x_val, y_val = load_fold("hold", select_features = False)

for i in range(7):
    m2  = pytorch_model(Model_2, "model_"+ str(i+100) +".pth", device)
    p2 = inference_fn(m2, x_val, device)
    l2 = log_loss_metric(y_val, p2)
    print(l2)
    all_preds.append(p2)

0.014986922622207879
0.014978571884187325
0.014976924231786438
0.014976544808136517
0.014987100500026593
0.014985886692641712
0.014976023817587879


## Now train an XGBoost model and blend with optuna 

[have a look pls](https://www.kaggle.com/fchmiel/xgboost-baseline-multilabel-classification)

[emergency source for 0.0185 something](https://www.kaggle.com/ksouriazer/the-power-of-blend)

In [ ]:
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline


In [ ]:
classifier = MultiOutputClassifier(XGBClassifier(tree_method='gpu_hist'))


clf = Pipeline([
                ('classify', classifier)
               ])

params = {'classify__estimator__colsample_bytree': 0.6522,
          'classify__estimator__gamma': 3.6975,
          'classify__estimator__learning_rate': 0.0503,
          'classify__estimator__max_delta_step': 2.0706,
          'classify__estimator__max_depth': 10,
          'classify__estimator__min_child_weight': 31.5800,
          'classify__estimator__n_estimators': 166,
          'classify__estimator__subsample': 0.8639
         }

_ = clf.set_params(**params)

In [ ]:
DATA_DIR = '/kaggle/input/lish-moa/'
test = pd.read_csv(DATA_DIR + 'test_features.csv')
x_test = test.iloc[:,1:].to_numpy()


num_folds = 7

for fold in range(num_folds):
    df = pd.read_csv("../input/lish-moa/train_features.csv")
    df = preprocess(df, select_features = True)

    targets_df = pd.read_csv('./train_targets_folds.csv')

    feature_columns = df.drop("sig_id", axis=1).columns
    target_columns = targets_df.drop(["sig_id", "kfold"], axis=1).columns


    df = df.merge(targets_df, on="sig_id", how="left")
    print(df.shape)
    df = df[df.kfold != "hold"]
    print(df.shape)

    train_df = df[df.kfold != str(fold)].reset_index(drop=True)
    valid_df = df[df.kfold == str(fold)].reset_index(drop=True)


    x_train = train_df[feature_columns].to_numpy()
    y_train = train_df[target_columns].to_numpy()

    x_val = valid_df[feature_columns].to_numpy()
    y_val = valid_df[target_columns].to_numpy()
    
    print("fitting...")
    clf.fit(x_train, y_train)
    
    print("done")
    val_preds = clf.predict_proba(x_val) # list of preds per class
    val_preds = np.array(val_preds)[:,:,1].T # take the positive class
    
    print("making preds...")
    preds = clf.predict_proba(x_test)
    all_preds.append(preds)
    
    print(log_loss_metric(y_val, val_preds))

In [65]:
!pip install optuna
clear_output()

In [37]:
import optuna

In [47]:
def scale_data(data, lower, upper):
    
    scaled = np.interp(data, (data.min(), data.max()), (lower, upper))
    return scaled 

def find_loss(params):
    
    weights_np = np.array(list(params.values()))
    blend_boi = blend(all_preds)
    sample_infer = blend_boi.predict(weights_np)
    
    
    loss = log_loss_metric(y_val, sample_infer)
    
    return loss


def objective(trial):

    params = {
        "w1": trial.suggest_uniform("w1", 0.0001, 1.0),
        "w2": trial.suggest_uniform("w2", 0.0001, 1.0),
        "w3": trial.suggest_uniform("w3", 0.0001, 1.0),
        "w4": trial.suggest_uniform("w4", 0.0001, 1.0),
        "w5": trial.suggest_uniform("w5", 0.0001, 1.0),
        "w6": trial.suggest_uniform("w6", 0.0001, 1.0),
        "w7": trial.suggest_uniform("w7", 0.0001, 1.0),
        "w8": trial.suggest_uniform("w8", 0.0001, 1.0),
        "w9": trial.suggest_uniform("w9", 0.0001, 1.0),
        "w10": trial.suggest_uniform("w10", 0.0001, 1.0),
        "w11": trial.suggest_uniform("w11", 0.0001, 1.0),
        "w12": trial.suggest_uniform("w12", 0.0001, 1.0),
        "w13": trial.suggest_uniform("w13", 0.0001, 1.0),
        "w14": trial.suggest_uniform("w14", 0.0001, 1.0)

    }

    loss_ = find_loss(params) 
    return loss_

In [48]:
study = optuna.create_study(direction="minimize")

[I 2020-10-06 07:01:27,740] A new study created in memory with name: no-name-ea1c492d-1718-4c5f-95d3-73ecad4664df


In [49]:
len(all_preds)

14

In [66]:
study.optimize(objective, n_trials=1000)

In [51]:
best_trial = study.best_params
best_trial

{'w1': 0.5755310193722791,
 'w2': 0.06050287895316951,
 'w3': 0.21168967336553382,
 'w4': 0.4499480072956117,
 'w5': 0.7087806565294461,
 'w6': 0.4963438527995536,
 'w7': 0.9995665803962417,
 'w8': 0.0006632508645747839,
 'w9': 0.06886520700497466,
 'w10': 0.018813631374325044,
 'w11': 0.0009756418602919041,
 'w12': 0.019041239665257285,
 'w13': 0.043206121299959196,
 'w14': 0.06900763852735275}

In [52]:
x_train, y_train, x_val, y_val = load_fold("hold", select_features = True)

blend_boi = blend(all_preds)
noice_weights = np.array(list(best_trial.values()))
sample_infer = blend_boi.predict(noice_weights)
loss = log_loss_metric(y_val, sample_infer)
print("loss on hold set: ", loss)

loss on hold set:  0.014533262113321026


In [53]:
list(best_trial.values())

[0.5755310193722791,
 0.06050287895316951,
 0.21168967336553382,
 0.4499480072956117,
 0.7087806565294461,
 0.4963438527995536,
 0.9995665803962417,
 0.0006632508645747839,
 0.06886520700497466,
 0.018813631374325044,
 0.0009756418602919041,
 0.019041239665257285,
 0.043206121299959196,
 0.06900763852735275]

In [54]:
df_w = pd.DataFrame(
    np.array(list(best_trial.values())).reshape(1,-1), 
    columns = list(best_trial.keys())
)

In [55]:
df_w.to_csv("blend_weights.csv", index = False)

In [56]:
df_w

,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14
0,0.575531,0.060503,0.21169,0.449948,0.708781,0.496344,0.999567,0.000663,0.068865,0.018814,0.000976,0.019041,0.043206,0.069008
